<a href="https://colab.research.google.com/github/jinwook8910/quant/blob/main/dart_web_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install yfinance

     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 6.4 MB 9.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
#다트 웹크롤링 강의
# 1. yfinance를 톻애 하이일드 회사채시수와 리츠인데스펀드 지수 크롤링

import pandas as pd
import numpy as np
import yfinance as yf
yf.pdr_override()

from pandas_datareader import data as pdr
pd.core.common.is_list_like = pd.api.types.is_list_like
import datetime

In [6]:
start = datetime.datetime(2008, 1, 1)
end = datetime.datetime(2022, 3, 31)

In [7]:
vwehx = pdr.get_data_yahoo('VWEHX', start, end)
vgsix = pdr.get_data_yahoo('VGSIX', start, end)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [8]:
vwehx_monthly = pd.DataFrame()
vgsix_monthly = pd.DataFrame()


In [13]:
for i in range(2008, 2023):
  for j in range(1, 13):
    try:
            vwehx_monthly = vwehx_monthly.append(vwehx[(vwehx.index.year == i) & (vwehx.index.month == j)].iloc[-1]) #iloc[-1] : 월말데이터 사용
            vgsix_monthly = vgsix_monthly.append(vgsix[(vgsix.index.year == i) & (vgsix.index.month == j)].iloc[-1])
    except IndexError: pass

# IndexError: single positional indexer is out-of-bounds  이 에러는 우리가 원하는 달마다                 

In [14]:
vwehx_monthly

,Open,High,Low,Close,Adj Close,Volume
2008-01-31,5.82,5.82,5.82,5.82,2.466034,0.0
2008-02-29,5.70,5.70,5.70,5.70,2.430057,0.0
2008-03-31,5.65,5.65,5.65,5.65,2.424592,0.0
2008-04-30,5.82,5.82,5.82,5.82,2.513115,0.0
2008-05-30,5.77,5.77,5.77,5.77,2.507576,0.0
...,...,...,...,...,...,...
2021-11-30,5.87,5.87,5.87,5.87,5.789867,0.0
2021-12-31,5.95,5.95,5.95,5.95,5.888569,0.0
2022-01-31,5.77,5.77,5.77,5.77,5.729293,0.0
2022-02-28,5.70,5.70,5.70,5.70,5.679751,0.0


In [16]:

#%% 2. 네이버금융에서 시계열 지수 크롤링
# 국채데이터를 크롤링해봅시다.


import pandas as pd
import numpy as np
from tqdm import tqdm # tqdm이 무엇인가??
import requests


# 네이버 금융 사이트에서 https://finance.naver.com/sise/sise_index.naver?code=KOSPI 에서
# https://finance.naver.com/item/sise_day.naver?code=114470 로 수정한다. item으로 바꿔주고, 114470 하이홀드 회사채.로 바꿔준다.
gov_3 = pd.DataFrame()
gov_url = 'https://finance.naver.com/item/sise_day.naver?code=114470'

#일별시세 1부터 50페이지 까지 한다.
for page in tqdm(range(1, 51)):
    pg_url = '{url}&page={page}'.format(url=gov_url, page = page) #넘겨주는 url이 계속 바뀔거니까, format 메서드로 그때그때 적용해주는 방식.
    gov_3 = gov_3.append(pd.read_html(requests.get(pg_url, headers={'User-agent': 'Mozilla/5.0'}).text)[0], ignore_index = True)
    # pandas에서 html을 불러오는 메서드 사용, 그 안에 requests로 요청부터 한다. 


100%|██████████| 50/50 [00:30<00:00,  1.62it/s]


In [26]:
gov_3

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022.05.10,108070.0,75.0,108060.0,108160.0,108060.0,39.0
2,2022.05.09,107995.0,325.0,107635.0,107995.0,107635.0,389.0
3,2022.05.06,107670.0,70.0,107600.0,107670.0,107600.0,373.0
4,2022.05.04,107600.0,215.0,107775.0,107775.0,107600.0,240.0
...,...,...,...,...,...,...,...
745,2020.05.08,113410.0,95.0,113330.0,113420.0,113330.0,234.0
746,2020.05.07,113315.0,30.0,113220.0,113315.0,113220.0,147.0
747,2020.05.06,113285.0,70.0,113215.0,113285.0,113200.0,246.0
748,2020.05.04,113215.0,110.0,113105.0,113240.0,113105.0,230.0


In [18]:

#%% 3. fnguide에서 코스피, 코스닥 전체 종목의 'ev에비따'와 '당기순이익'을 크롤링해봅시다. 


company = pd.read_csv('company_kospi.csv', encoding='cp949')

In [21]:

import pandas as pd
from bs4 import BeautifulSoup
from urllib import request as rq
from tqdm import tqdm

# 원래는 이 company_kospi도 크롤링해서 만들어야 한다.
company100 = company.iloc[:100, :]

company100

,분류,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,구글코드
0,코스피,두산우,155,회사 본부 및 경영 컨설팅 서비스업,"청주,소주,인쇄회로기판(인쇄회로용동박적층판),관련전자부품,기계 제조,판매/출판사업,...",1973-06-29,12월,"박정원, 이재경",http://www.doosancorp.co.kr,서울특별시,KRX:000155
1,코스피,한화3우B,00088K,기타 화학제품 제조업,"화약,화공품,자동선반류,공작기계 제조,전자교환기,건설,정보통신,종합무역/임직원 및 ...",1976-06-24,12월,"최양수, 이태종, 김연철, 이민석",http://www.hanwhacorp.co.kr,서울특별시,KRX:00088K
2,코스피,S-Oil우,10955,석유 정제품 제조업,"석유제품,가스,윤활기유,윤활유,그리스,석유화학제품 제조,도매",1987-05-27,12월,Othman Al-Ghamdi,http://www.s-oil.com,서울특별시,KRX:010955
3,코스피,LG화학우,51915,기초 화학물질 제조업,"유화/기능/합성수지,재생섬유소,산업재,리튬이온전지,평광판,PVC 제조,도매",2001-04-25,12월,박진수,http://www.lgchem.com,서울특별시,KRX:051915
4,코스피,한국금융지주우,71055,기타 금융업,금융지주회사,2003-07-21,12월,김남구,http://www.koreaholdings.com,서울특별시,KRX:071055
...,...,...,...,...,...,...,...,...,...,...,...
95,코스피,바다로19호,155900,운송장비 임대업,NaN,2012-06-29,12월,이재붕,NaN,제주특별자치도,KRX:155900
96,코스피,사조씨푸드,14710,수산물 가공 및 저장 처리업,횟감용 참치 및 기타 일반수산물,2012-06-29,12월,최창욱,http://www.sajosf.co.kr,서울특별시,KRX:014710
97,코스피,하이골드3호,153360,운송장비 임대업,NaN,2012-05-04,12월,오운용,NaN,제주특별자치도,KRX:153360
98,코스피,코오롱머티리얼,144620,화학섬유 제조업,"합성섬유 제조/직물 도매,무역",2012-04-05,12월,대표이사 사장 최석순,http://www.kolonfm.com,경기도,KRX:144620


In [22]:

# INDEX_URL에는 컴퍼니 가이드 사이트의 투자지표에 에비에빗다정보를 넣기 위함이다. FS_URL 재무제표를 넣는다.
INDEX_URL = 'http://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'
FS_URL = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'

sample_code = '005930'

## get ev/ebitda
index_html = rq.urlopen(INDEX_URL.format(sample_code)).read() #바이트형태로 소스 가져옴
index_soup = BeautifulSoup(index_html, 'html.parser') #바이트를 html로 파싱해서 정리해서 보여줌


In [23]:

# id를 통해 찾는게 가장 쉬움. 한번밖에 사용이 안되기때문
# tr태크에서 id를 찾아본다. id는 하나밖에 없기 때문, p_grid1_14인걸 알았고, 값을 저장하는 태그인 td를 추가로 찾는다.
ev_cells = index_soup.find('tr',{'id':'p_grid1_14'}).find_all('td') #해당부분 html코드를 뜯어옴
ev = float(ev_cells[3].string) #지표를 ev란 변수에 할당, 원하는 지표를 []로 찾아서 float형으로 변환.

#%% 당기순이익 크롤링
fs_html = rq.urlopen(FS_URL.format(sample_code)).read()
fs_soup = BeautifulSoup(fs_html, 'html.parser') # 똑같이 BeautifulSoup로 아름답게 파싱해줍니다.
# 아까와 같이 id태그를 찾아야함! 크롤링 방식은 이것만 있는게 아니다. 다양한 방법을 고려하자. 이거보다 나은게..
is_cells = fs_soup.find('div', {'id':'divSonikY'}).find_all('tr', {'class':'rwf'})
# id를 divSonikY를 찾았고 tr로 가져온다.

#income statement sells
is_cells[-3].find_all('td')[3]
ni = float(is_cells[-3].find_all('td')[3].string.replace(',','')) # replace로 숫자 사이 ,제거 후 float으로 형변환
# 크롤링 방식이 좀 주먹구구 인것 같긴하다 원리를 잘 알아야할듯.

#%% 크롤링함수 만들기   
#li라는 (임의의)파라미터를 사용  #예외처리(for안에)는 젤 마지막에  #tqdm도 젤 마지막에
code_list = company100['종목코드'].dropna()
code_list

0        155
1     00088K
2      10955
3      51915
4      71055
       ...  
95    155900
96     14710
97    153360
98    144620
99    152550
Name: 종목코드, Length: 99, dtype: object

In [24]:

def crawling(li):
    result = {}
    
    for code in tqdm(li):
        try:
        #get url 아까 가져온 에비에빗다와 재무제표 url이다.
            INDEX_URL = 'http://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'
            FS_URL = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'
            
            #get company name
            name = company[company['종목코드'] == code]['회사명'].values[0]
            
            ##get evebitda (짜논 코드 복붙)
            index_html = rq.urlopen(INDEX_URL.format(code)).read() #바이트형태로 소스 가져옴
            BeautifulSoup(index_html, 'html.parser') #바이트를 html로 파싱해서 정리해서 보여줌
            index_soup = BeautifulSoup(index_html, 'html.parser')
            #id를 통해 찾는게 가장 쉬움. 한번밖에 사용이 안되기때문
            ev_cells = index_soup.find('tr', {'id':'p_grid1_14'}).find_all('td') #해당부분 html코드를 뜯어옴
            ev = float(ev_cells[3].string) #이비에비타 지표를 ev란 변수에 할당
            
            ##get 당기순이익
            fs_html = rq.urlopen(FS_URL.format(code)).read() #sample코드가 아니라 code로 대체
            fs_soup = BeautifulSoup(fs_html, 'html.parser')
            # id태그를 찾아야함!
            is_cells = fs_soup.find('div', {'id':'divSonikY'}).find_all('tr', {'class':'rwf'})
            #income statement sells
            is_cells[-3].find_all('td')[3]
            ni = float(is_cells[-3].find_all('td')[3].string.replace(',',''))
            
            # 이렇게 dictionary 형식으로 정리해서 결과를 저장한다.
            result[name] = [code, ev, ni] #for를 돌면서 다 저장되게. (빈 result 설정은 for 위에서, 저장명령은 안에서)

        # 종목이 다를때 일어나는 에러 등등을 무시해버리는 try except 구문을 사용한다.
        # 좋은 코드라고 생각하지는 않는다. 파이썬은 문자를 패스하지 않는다. 우리는 상관없는데.
        # 파이썬이 멍청하고 나는 괜찮다라는 확신이 있을때 이 except를 사용한다.
        # 보안에 취약할 수 있으니 주의해서 사용해야 한다.
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
        
    # 딕셔너리를 df로 만들기. 당연히 for 밖에 위치함
    result = pd.DataFrame(result)
    result = result.transpose()
    column_names = ['Code', 'EV/EBITDA', 'NI']    
    result.columns = column_names
    
    return result # 함수 선언 완료!

#%% 함수 돌려보자
result_df = crawling(code_list)

100%|██████████| 99/99 [03:51<00:00,  2.34s/it]


In [25]:
result_df

,Code,EV/EBITDA,NI
제일약품,271980,43.99,-150.0
오리온,271560,9.65,2637.0
미원에스씨,268280,8.11,799.0
경동도시가스,267290,2.52,222.0
넷마블게임즈,251270,30.05,2492.0
현대건설기계,267270,7.19,1264.0
현대로보틱스,267250,92.9,1860.0
현대일렉트릭,267260,7.26,-337.0
크라운제과,264900,5.09,131.0
덴티움,145720,11.67,555.0
